In [121]:
import parselmouth
import textgrids
from matplotlib import pyplot
from parselmouth.praat import call

In [122]:
# les phrases au choix pour synthétiser
phrase_choix = input("Voulez-vous synthétiser quelle phrase :\n 1 : Le cours de master un plurital de monsieur gendrot est très intéressant \n 2 : Le cours de master un plurital de madame taravella est très difficile \n 3: Je suis un étudiant chinois qui vient de Paris trois \n 4: L'acquisition du langage chez l'enfant est un sujet intéressant \n 5: Les enfants qui jouent avec les chiens sont très mignons")
phrases = {"1":"Le cours de master un plurital de monsieur gendrot est très intéressant",
           "2":"Le cours de master un plurital de madame taravella est très difficile",
           "3":"Je suis un étudiant chinois qui vient de Paris trois",
           "4":"L'acquisition du langage chez l'enfant est un sujet intéressant",
           "5":"Les enfants qui jouent avec les chiens sont très mignons"}
phrase = phrases[phrase_choix]

In [123]:
son = 'logatomes.wav'
grille = 'logatomes.TextGrid'
chemin_synthese = f"./phrase{phrase_choix}.wav"

# ouvrir le son
sound = parselmouth.Sound(son)
segmentation = textgrids.TextGrid(grille)

#lire les diphones
diphones = segmentation['Diphones']

#rendre les valeurs de diphones dans un dictionnaires afin de mieux saisir les diphones cibles à l'avenir
dic_diphones = {}
for a in range(len(diphones)-1):
    if diphones[a].text != "" and diphones[a+1].text != "": #chercher les diphones
        milieu_phoneme1 = (diphones[a].xmin + diphones[a].xmax)/2
        #cherher le point de croisement zéro le plus proche de son donné
        milieu_phoneme1 = sound.get_nearest_zero_crossing(milieu_phoneme1,1)
        
        milieu_phoneme2 = (diphones[a+1].xmin + diphones[a+1].xmax)/2
        milieu_phoneme2 = sound.get_nearest_zero_crossing(milieu_phoneme2,1)
        
        # extraire les sons de diphone
        extrait = sound.extract_part(milieu_phoneme1, milieu_phoneme2, parselmouth.WindowShape.RECTANGULAR, 1, False)
        


        allongement = 1.5 # choisir allonger ou accélérer le son
        
        #manipuler l'hauteur et la durée
        manipulation = call(extrait, "To Manipulation", 0.01, 80.43, 600) 
        duration_tier = call(manipulation, "Extract duration tier")  
        call(duration_tier, "Remove points between", 0,extrait.duration) 
        call(duration_tier,"Add point", extrait.duration/2,allongement)
        call([manipulation,duration_tier],"Replace duration tier") 
        extrait = call(manipulation,"Get resynthesis (overlap-add)")
        
        # créer un dictionnaire pour stoker les diphones comme clés，les sons comme valeurs
        dic_diphones[diphones[a].text + diphones[a+1].text] = extrait


print(dic_diphones)



{'l@': <parselmouth.Sound object at 0x11a68ccf0>, 'd@': <parselmouth.Sound object at 0x10e4fe930>, 'ma': <parselmouth.Sound object at 0x10e4feb30>, 'tE': <parselmouth.Sound object at 0x10e4feab0>, 'R9~': <parselmouth.Sound object at 0x11a95f1f0>, 'ly': <parselmouth.Sound object at 0x10aaf6bb0>, 'Ri': <parselmouth.Sound object at 0x10d817230>, 'ta': <parselmouth.Sound object at 0x11a0aebb0>, 'm@': <parselmouth.Sound object at 0x10da6ce30>, 'j2': <parselmouth.Sound object at 0x10da6cdf0>, 'ZA~': <parselmouth.Sound object at 0x10e4fdf70>, 'Ro': <parselmouth.Sound object at 0x10e4fe330>, 'RE': <parselmouth.Sound object at 0x10e4fe670>, 'zE~': <parselmouth.Sound object at 0x10e4fe8b0>, 'Re': <parselmouth.Sound object at 0x10e4fe9f0>, 'sA~': <parselmouth.Sound object at 0x10e4fe870>, 'da': <parselmouth.Sound object at 0x10e4fea70>, 'Ra': <parselmouth.Sound object at 0x10e4fe230>, 'vE': <parselmouth.Sound object at 0x10e4fe5b0>, 'la': <parselmouth.Sound object at 0x10e4fe470>, 'di': <parselmo

In [124]:
# insérer le dictionnaire de phonétique pour obtenir les phonèmes de phrases
dico = {}
with open('dico_UTF8.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split('\t') 
        dico[key] = value
print(dico['un'])


#obtenir phonétique de la phrase à l'aide de dico_phonétique
phrase_phonetique = []
# transforme cette phrase en phonétique
for key in phrase.split():
    phrase_phonetique.append(dico[key])
phrase_phonetique = "".join(phrase_phonetique)

print(phrase_phonetique)

9~
lezA~fA~kiZuavEkleSjE~sO~tREmiNO~


In [125]:
# créer une fonction pour obtenir la liste de diphones
# NB : pour certains phonemes comme A~, E~ etc, ils contient 2 caractères
def extract_diphones(sequence):
    phonemes = []
    i = 0

    # pourcourir phrase_phonetique
    while i < len(sequence):
        # Vérifier les phonèmes se terminant par `~``
        if i + 1 < len(sequence) and sequence[i + 1] == '~':
            phonemes.append(sequence[i:i + 2])
            i += 2
        else:
            phonemes.append(sequence[i])
            i += 1

    # créer les diphones à partir des phonemes
    diphones = []
    for j in range(len(phonemes) - 1):
        diphones.append(phonemes[j] + phonemes[j + 1])

    return diphones

# obtenir les diphones à l'aide de cette fonction
diphones_list = extract_diphones(phrase_phonetique)
print(diphones_list)


['le', 'ez', 'zA~', 'A~f', 'fA~', 'A~k', 'ki', 'iZ', 'Zu', 'ua', 'av', 'vE', 'Ek', 'kl', 'le', 'eS', 'Sj', 'jE~', 'E~s', 'sO~', 'O~t', 'tR', 'RE', 'Em', 'mi', 'iN', 'NO~']


In [126]:
#obtenir la liste des sons pour concaténer à partir de dico_diphones 
son_list=[]
for i in range(len(diphones_list)):
    son=dic_diphones[diphones_list[i]]
    son_list.append(son)

print(son_list)    

[<parselmouth.Sound object at 0x10fe4e5f0>, <parselmouth.Sound object at 0x11a7d24b0>, <parselmouth.Sound object at 0x11a7d2d30>, <parselmouth.Sound object at 0x11aa06d70>, <parselmouth.Sound object at 0x10fe4d830>, <parselmouth.Sound object at 0x10da82630>, <parselmouth.Sound object at 0x10fe4dbb0>, <parselmouth.Sound object at 0x10da826b0>, <parselmouth.Sound object at 0x10fe4e670>, <parselmouth.Sound object at 0x10d8e2470>, <parselmouth.Sound object at 0x10fe4d870>, <parselmouth.Sound object at 0x10e4fe5b0>, <parselmouth.Sound object at 0x10da82770>, <parselmouth.Sound object at 0x11aa07270>, <parselmouth.Sound object at 0x10fe4e5f0>, <parselmouth.Sound object at 0x10da82830>, <parselmouth.Sound object at 0x11aa070b0>, <parselmouth.Sound object at 0x10fe4d8f0>, <parselmouth.Sound object at 0x11a7d2a70>, <parselmouth.Sound object at 0x10fe4ea70>, <parselmouth.Sound object at 0x11a7d2b70>, <parselmouth.Sound object at 0x11aa06eb0>, <parselmouth.Sound object at 0x10e4fe670>, <parselmou

In [127]:
# concaténer les sons et enregistrer
son_concatene = parselmouth.praat.call(son_list, "Concatenate")
son_concatene.save(chemin_synthese, "WAV")
